In [ ]:
pip install pingouin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7048 sha256=7566403b67c7337435c42b7624c127ad7de0667b6f518cfc7a6027ad348e0f0e
  Stored in directory: /root/.cache/pip/wheels/3d/fe/b0/27a9892da57472e538c7452a721a9cf463cc03cf7379889266
Successfully built littleutils


In [ ]:
pip install scikit-posthocs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install flirt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.1 MB/s eta 0:00:00


In [ ]:
import seaborn as sns
import flirt.reader.empatica
import flirt.with_
import matplotlib.pyplot as plt
import matplotlib as mpl
# import latex
import os
import csv
from glob import glob
import copy
import random
import pickle
import numpy as np
from scipy import stats
import scipy.io.wavfile
import scipy.signal
import pingouin as pg
import pandas as pd

def cliffs_delta(lst1, lst2):
    """Calculates Cliff's Delta, a non-parametric measure of effect size for two samples."""
    n1 = len(lst1)
    n2 = len(lst2)
    lst1 = sorted(lst1)
    lst2 = sorted(lst2)
    i, j = 0, 0
    delta = 0
    while i < n1 and j < n2:
        if lst1[i] <= lst2[j]:
            delta += j
            i += 1
        else:
            j += 1
    delta = delta / (n1 * n2)
    return delta
    
def cohen_d(x,y):
    nx = len(x)
    ny = len(y)
    dof = nx + ny - 2
    return (np.mean(x) - np.mean(y)) / np.sqrt(((nx-1)*np.std(x, ddof=1) ** 2 + (ny-1)*np.std(y, ddof=1) ** 2) / dof)


In [ ]:
!unzip "/content/Physiological Signal.zip" -d "/content/"
!unzip "/content/Physiological Signal/Dirt Rally.zip" -d "/content/"
!unzip "/content/Physiological Signal/Warthunder.zip" -d "/content/"
!unzip "/content/Physiological Signal/Minecraft.zip" -d "/content/"
!unzip "/content/Physiological Signal/Phasmophobia.zip" -d "/content/"

#Heart Rate

In [ ]:
paths = ["/content/Phasmophobia", "/content/Warthunder", "/content/Minecraft", "/content/Dirt Rally"]
hr = [[[], [], [], []], [[], [], [], []]]
hr_bs = [[], [], [], []]
hr_bs_std=[[], [], [], []]
i=-1
for path in paths:
  i+=1
  for participant in glob(path + "/*/PC", recursive = True):
    p = glob(participant+"/HR.csv")[0]
    hr[0][i].append(np.mean(flirt.reader.empatica.read_hr_file_into_df(p)["hr"][100:-10]).tolist())
  for participant in glob(path + "/*/VR", recursive = True):
    p = glob(participant+"/HR.csv")[0]
    hr[1][i].append(np.mean(flirt.reader.empatica.read_hr_file_into_df(p)["hr"][100:-10]).tolist())
  for participant in glob(path + "/*/Baseline", recursive = True):
    p = glob(participant+"/HR.csv")[0]
    # print(p)
    hr_bs[i].append(np.mean(flirt.reader.empatica.read_hr_file_into_df(p)["hr"][100:-10]).tolist())
    hr_bs_std[i].append(np.std(flirt.reader.empatica.read_hr_file_into_df(p)["hr"][100:-10]).tolist())

In [ ]:
hr_norm = [[[], [], [], []], [[], [], [], []]]
for i in range(4):
  # print(i)
  hr_norm[0][i] = (np.array(hr[0][i]) - np.array(hr_bs[i]))
  hr_norm[1][i] = (np.array(hr[1][i]) - np.array(hr_bs[i]))
# print(hr_norm[0])

##Mean Values

In [ ]:
Games = ['LVHA', 'HVLA', 'LVLA', 'HVHA']
Type = ['FS', 'VR']

arr=[]
i=0
for x in hr:
  tmp=[]
  j=0
  # print(len(x))
  for y in x:
    df = pd.DataFrame(y, columns = ["Heart Rate"])
    df["CMA Quadrant"] = Games[j]
    df["Type"] = Type[i]
    tmp.append(df)
    j+=1
  arr.append(pd.concat(tmp))
  i+=1

df = pd.concat(arr, ignore_index = True)
print(df)
df.to_csv("HR_Boxplot.csv")

    Heart Rate CMA Quadrant Type
0    85.312029         LVHA   FS
1    80.658842         LVHA   FS
2    79.074062         LVHA   FS
3    84.249643         LVHA   FS
4    89.329963         LVHA   FS
..         ...          ...  ...
61   88.675596         HVHA   VR
62   73.974738         HVHA   VR
63   62.195039         HVHA   VR
64   61.180222         HVHA   VR
65   83.479615         HVHA   VR

[66 rows x 3 columns]


In [ ]:
pc = df.query('Type=="FS"')["Heart Rate"].tolist()
vr = df.query('Type=="VR"')["Heart Rate"].tolist()

print("HR")
print(f"FS: mu: {np.mean(pc):.2f}, sigma: {np.std(pc):.2f}")
print(f"VR: mu: {np.mean(vr):.2f}, sigma: {np.std(vr):.2f}")

HR
FS: mu: 77.27, sigma: 9.92
VR: mu: 81.52, sigma: 10.71


##Comparing HR

In [ ]:
Games = ['LVHA', 'HVLA', 'LVLA', 'HVHA']
Type = ['FS', 'VR']

arr=[]
i=0
for x in hr_norm:
  tmp=[]
  j=0
  # print(len(x))
  for y in x:
    df = pd.DataFrame(y, columns = ["Normalised HR"])
    df["Type"] = Type[i]
    tmp.append(df)
    j+=1
  # print(j)
  arr.append(pd.concat(tmp))
  i+=1
# print(i)

df = pd.concat(arr, ignore_index = True)
print(df)
df.to_csv("HR.csv")

    Normalised HR Type
0       -4.275331   FS
1      -17.661158   FS
2       -2.864298   FS
3       -8.306594   FS
4       -3.684550   FS
..            ...  ...
61      29.650318   VR
62      -3.723309   VR
63       7.298882   VR
64      -2.390406   VR
65       0.631050   VR

[66 rows x 2 columns]


In [ ]:
pc = df.query('Type=="FS"')["Normalised HR"].tolist()
vr = df.query('Type=="VR"')["Normalised HR"].tolist()

normalancy_pc = stats.shapiro(pc)
normalancy_vr = stats.shapiro(vr)
print(f"FS: {normalancy_pc}")
print(f"VR: {normalancy_vr}")

result = stats.ttest_rel(pc, vr)
print(result)

print(f'Cohen-d: {cohen_d(pc, vr):.2f}')
print(f'Cliffs: {cliffs_delta(pc, vr):.2f}')

FS: ShapiroResult(statistic=0.9776724576950073, pvalue=0.7138577699661255)
VR: ShapiroResult(statistic=0.9812292456626892, pvalue=0.8226732611656189)
TtestResult(statistic=-2.536701331638034, pvalue=0.016266536026586027, df=32)
Cohen-d: -0.41
Cliffs: 0.40


#LF/HF Value

In [ ]:
paths = ["/content/Phasmophobia", "/content/Warthunder", "/content/Minecraft", "/content/Dirt Rally"]
lf=[[[], [], [], []],  [[], [], [], []],  [[], [], [], []]]
hf=[[[], [], [], []],  [[], [], [], []],  [[], [], [], []]]
i=0
for path in paths:
  for participant in glob(path + "/*/PC", recursive = True):
    p = glob(participant+"/IBI.csv")[0]
    print(p)
    try:
      ibis = flirt.reader.empatica.read_ibi_file_into_df(p)
      hrv_features = flirt.get_hrv_features(ibis['ibi'])
      if(len(hrv_features.columns)>5):
        lf[0][i].append(hrv_features['hrv_lf'])
        hf[0][i].append(hrv_features['hrv_hf'])
    except:
      print("error")
  for participant in glob(path + "/*/VR", recursive = True):
    p = glob(participant+"/IBI.csv")[0]
    print(p)
    try:
      ibis = flirt.reader.empatica.read_ibi_file_into_df(p)
      hrv_features = flirt.get_hrv_features(ibis['ibi'])
      if(len(hrv_features.columns)>5):
        lf[1][i].append(hrv_features['hrv_lf'])
        hf[1][i].append(hrv_features['hrv_hf'])
    except:
      print("error")
  i+=1

/content/Phasmophobia/28/PC/IBI.csv


HRV features: 100%|██████████| 485/485 [00:13<00:00, 35.06it/s] 


/content/Phasmophobia/32/PC/IBI.csv


HRV features: 100%|██████████| 826/826 [00:02<00:00, 411.92it/s]


/content/Phasmophobia/29/PC/IBI.csv


...

/content/Dirt Rally/20/VR/IBI.csv


HRV features: 100%|██████████| 299/299 [00:01<00:00, 209.82it/s]


/content/Dirt Rally/21/VR/IBI.csv


HRV features: 100%|██████████| 216/216 [00:01<00:00, 191.74it/s]


/content/Dirt Rally/19/VR/IBI.csv
error


In [ ]:
data = [[[], []], [[], []], [[], []]]
i=0
for x in lf[:-1]:
  j=0
  for y in x:
    tmp=[]
    k=0
    for z in y:
      tmp.append(np.mean(z.dropna().tolist()))
      k+=1
    data[0][i].append(tmp)
  i+=1
i=0
for x in hf[:-1]:
  j=0
  for y in x:
    tmp=[]
    for z in y:
      tmp.append(np.mean(z.dropna().tolist()))
    data[1][i].append(tmp)
    j+=1
  i+=1

for i in [0, 1]:
  for x,y in zip(data[0][i], data[1][i]):
    data[2][i]+=(np.array(np.array(x)/np.array(y))).tolist()


print("LF/HF")
vr = data[2][1][:25]
pc = data[2][0][:25]
print(f"FS: Mdn: {np.median(pc):.2f} ({np.percentile(pc, 25):.2f}-{np.percentile(pc, 75):.2f})")
print(f"VR: Mdn: {np.median(vr):.2f} ({np.percentile(vr, 25):.2f}-{np.percentile(vr, 75):.2f})")
print(f"FS: $\mu: $ {np.mean(pc):.2f}, $\sigma: $: {np.std(pc):.2f}")
print(f"VR: $\mu: $ {np.mean(vr):.2f}, $\sigma: $: {np.std(vr):.2f}")
print(stats.shapiro(pc))
print(stats.shapiro(vr))
result = stats.ttest_rel(pc, vr)
print(result)
print(cohen_d(pc, vr))

LF/HF
FS: Mdn: 1.43 (0.93-1.67)
VR: Mdn: 1.30 (0.86-1.91)
FS: $\mu: $ 1.36, $\sigma: $: 0.61
VR: $\mu: $ 1.43, $\sigma: $: 0.63
ShapiroResult(statistic=0.9464111924171448, pvalue=0.20784859359264374)
ShapiroResult(statistic=0.9661115407943726, pvalue=0.5489051342010498)
TtestResult(statistic=-0.5036821429215032, pvalue=0.6190769097581027, df=24)
-0.10670792611193591


#EDA

In [ ]:
#@title cvxEDA Algorith
"""
______________________________________________________________________________

 File:                         cvxEDA.py
 Last revised:                 07 Nov 2015 r69
 ______________________________________________________________________________

 Copyright (C) 2014-2015 Luca Citi, Alberto Greco
 
 This program is free software; you can redistribute it and/or modify it under
 the terms of the GNU General Public License as published by the Free Software
 Foundation; either version 3 of the License, or (at your option) any later
 version.
 
 This program is distributed in the hope that it will be useful, but WITHOUT
 ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS
 FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.
 
 You may contact the author by e-mail (lciti@ieee.org).
 ______________________________________________________________________________

 This method was first proposed in:
 A Greco, G Valenza, A Lanata, EP Scilingo, and L Citi
 "cvxEDA: a Convex Optimization Approach to Electrodermal Activity Processing"
 IEEE Transactions on Biomedical Engineering, 2015
 DOI: 10.1109/TBME.2015.2474131

 If you use this program in support of published research, please include a
 citation of the reference above. If you use this code in a software package,
 please explicitly inform the end users of this copyright notice and ask them
 to cite the reference above in their published research.
 ______________________________________________________________________________
"""

import numpy as np
import cvxopt as cv
import cvxopt.solvers

def cvxEDA(y, delta, tau0=2., tau1=0.7, delta_knot=10., alpha=8e-4, gamma=1e-2,
           solver=None, options={'reltol':1e-9}):
    """CVXEDA Convex optimization approach to electrodermal activity processing

    This function implements the cvxEDA algorithm described in "cvxEDA: a
    Convex Optimization Approach to Electrodermal Activity Processing"
    (http://dx.doi.org/10.1109/TBME.2015.2474131, also available from the
    authors' homepages).
    

    Arguments:
       y: observed EDA signal (we recommend normalizing it: y = zscore(y))
       delta: sampling interval (in seconds) of y
       tau0: slow time constant of the Bateman function
       tau1: fast time constant of the Bateman function
       delta_knot: time between knots of the tonic spline function
       alpha: penalization for the sparse SMNA driver
       gamma: penalization for the tonic spline coefficients
       solver: sparse QP solver to be used, see cvxopt.solvers.qp
       options: solver options, see:
                http://cvxopt.org/userguide/coneprog.html#algorithm-parameters

    Returns (see paper for details):
       r: phasic component
       p: sparse SMNA driver of phasic component
       t: tonic component
       l: coefficients of tonic spline
       d: offset and slope of the linear drift term
       e: model residuals
       obj: value of objective function being minimized (eq 15 of paper)
    """

    n = len(y)
    y = cv.matrix(y)

    # bateman ARMA model
    a1 = 1./min(tau1, tau0) # a1 > a0
    a0 = 1./max(tau1, tau0)
    ar = np.array([(a1*delta + 2.) * (a0*delta + 2.), 2.*a1*a0*delta**2 - 8.,
        (a1*delta - 2.) * (a0*delta - 2.)]) / ((a1 - a0) * delta**2)
    ma = np.array([1., 2., 1.])

    # matrices for ARMA model
    i = np.arange(2, n)
    A = cv.spmatrix(np.tile(ar, (n-2,1)), np.c_[i,i,i], np.c_[i,i-1,i-2], (n,n))
    M = cv.spmatrix(np.tile(ma, (n-2,1)), np.c_[i,i,i], np.c_[i,i-1,i-2], (n,n))

    # spline
    delta_knot_s = int(round(delta_knot / delta))
    spl = np.r_[np.arange(1.,delta_knot_s), np.arange(delta_knot_s, 0., -1.)] # order 1
    spl = np.convolve(spl, spl, 'full')
    spl /= max(spl)
    # matrix of spline regressors
    i = np.c_[np.arange(-(len(spl)//2), (len(spl)+1)//2)] + np.r_[np.arange(0, n, delta_knot_s)]
    nB = i.shape[1]
    j = np.tile(np.arange(nB), (len(spl),1))
    p = np.tile(spl, (nB,1)).T
    valid = (i >= 0) & (i < n)
    B = cv.spmatrix(p[valid], i[valid], j[valid])

    # trend
    C = cv.matrix(np.c_[np.ones(n), np.arange(1., n+1.)/n])
    nC = C.size[1]

    # Solve the problem:
    # .5*(M*q + B*l + C*d - y)^2 + alpha*sum(A,1)*p + .5*gamma*l'*l
    # s.t. A*q >= 0

    old_options = cv.solvers.options.copy()
    cv.solvers.options.clear()
    cv.solvers.options.update(options)
    if solver == 'conelp':
        # Use conelp
        z = lambda m,n: cv.spmatrix([],[],[],(m,n))
        G = cv.sparse([[-A,z(2,n),M,z(nB+2,n)],[z(n+2,nC),C,z(nB+2,nC)],
                    [z(n,1),-1,1,z(n+nB+2,1)],[z(2*n+2,1),-1,1,z(nB,1)],
                    [z(n+2,nB),B,z(2,nB),cv.spmatrix(1.0, range(nB), range(nB))]])
        h = cv.matrix([z(n,1),.5,.5,y,.5,.5,z(nB,1)])
        c = cv.matrix([(cv.matrix(alpha, (1,n)) * A).T,z(nC,1),1,gamma,z(nB,1)])
        res = cv.solvers.conelp(c, G, h, dims={'l':n,'q':[n+2,nB+2],'s':[]})
        obj = res['primal objective']
    else:
        # Use qp
        Mt, Ct, Bt = M.T, C.T, B.T
        H = cv.sparse([[Mt*M, Ct*M, Bt*M], [Mt*C, Ct*C, Bt*C], 
                    [Mt*B, Ct*B, Bt*B+gamma*cv.spmatrix(1.0, range(nB), range(nB))]])
        f = cv.matrix([(cv.matrix(alpha, (1,n)) * A).T - Mt*y,  -(Ct*y), -(Bt*y)])
        res = cv.solvers.qp(H, f, cv.spmatrix(-A.V, A.I, A.J, (n,len(f))),
                            cv.matrix(0., (n,1)), solver=solver)
        obj = res['primal objective'] + .5 * (y.T * y)
    cv.solvers.options.clear()
    cv.solvers.options.update(old_options)

    l = res['x'][-nB:]
    d = res['x'][n:n+nC]
    t = B*l + C*d
    q = res['x'][:n]
    p = A * q
    r = M * q
    e = y - r - t

    return [r, p, t, l, d, e, obj]

In [ ]:
paths = ["/content/Phasmophobia", "/content/Warthunder", "/content/Minecraft", "/content/Dirt Rally"]
eda_PC = []
eda_VR = []
eda_BS = [[], [], []]
for path in paths:
  for participant in glob(path + "/*/Baseline", recursive = True):
    p = glob(participant+"/EDA.csv")[0]
    eda_BS[0].append(flirt.reader.empatica.read_eda_file_into_df(p)["eda"])
    eda_BS[1].append(np.mean(flirt.reader.empatica.read_eda_file_into_df(p)["eda"]))
    eda_BS[2].append(np.std(flirt.reader.empatica.read_eda_file_into_df(p)["eda"]))
  for participant in glob(path + "/*/PC", recursive = True):
    p = glob(participant+"/EDA.csv")[0]
    eda_PC.append(flirt.reader.empatica.read_eda_file_into_df(p)["eda"])
  for participant in glob(path + "/*/VR", recursive = True):
    p = glob(participant+"/EDA.csv")[0]
    eda_VR.append(flirt.reader.empatica.read_eda_file_into_df(p)["eda"])

In [ ]:
# 0: Phasic, SCR
# 1: Tonic, SCL
cvx_eda_PC = [[], [], [], []]
cvx_eda_VR = [[], [], [], []]
cvx_eda_BS = [[], [], [], []]
# 0 - 8  : Phasmophobia
# 8 - 15 : Warthunder
# 15 - 24: Minecraft
# 24 - 33: Dirt Rally
i=0
j=0
for eda in eda_BS[0]:
  cvx = cvxEDA(eda, 4)
  cvx_eda_BS[j].append([cvx[0], cvx[2]])
  i+=1
  if(i==8 or i==15 or i==24):
    j+=1
i=0
j=0
for eda in eda_PC:
  cvx = cvxEDA(eda, 4)
  cvx_eda_PC[j].append([cvx[0], cvx[2]])
  i+=1
  if(i==8 or i==15 or i==24):
    j+=1

i=0
j=0
for eda in eda_VR:
  cvx = cvxEDA(eda, 4)
  cvx_eda_VR[j].append([cvx[0], cvx[2]])
  i+=1
  if(i==8 or i==15 or i==24):
    j+=1

     pcost       dcost       gap    pres   dres
 0: -5.8626e+04 -5.8625e+04  1e+03  4e+01  9e-03
 1: -5.8626e+04 -5.8732e+04  1e+02  3e+00  7e-04
 2: -5.8626e+04 -5.8634e+04  9e+00  2e-01  5e-05
 3: -5.8626e+04 -5.8627e+04  1e+00  9e-03  2e-06
 4: -5.8626e+04 -5.8627e+04  2e-01  2e-03  4e-07
 5: -5.8627e+04 -5.8627e+04  5e-02  2e-04  4e-08
 6: -5.8627e+04 -5.8627e+04  1e-02  2e-05  4e-09
 7: -5.8627e+04 -5.8627e+04  2e-03  3e-06  7e-10
 8: -5.8627e+04 -5.8627e+04  5e-04  5e-07  1e-10
 9: -5.8627e+04 -5.8627e+04  9e-05  6e-08  1e-11
10: -5.8627e+04 -5.8627e+04  7e-06  3e-09  6e-13
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -1.6662e+04 -1.6661e+04  1e+03  4e+01  2e-02
 1: -1.6661e+04 -1.6766e+04  1e+02  3e+00  1e-03
 2: -1.6661e+04 -1.6669e+04  7e+00  2e-01  7e-05
 3: -1.6662e+04 -1.6663e+04  9e-01  7e-03  3e-06
 4: -1.6662e+04 -1.6663e+04  2e-01  1e-03  6e-07
 5: -1.6662e+04 -1.6662e+04  6e-02  2e-04  9e-08
 6: -1.6662e+04 -1.6662e+04  1e-02  3e-05  1e-0

In [ ]:
# 0: Phasic, SCR
# 1: Tonic, SCL
# 0 - 8  : Phasmophobia
# 8 - 15 : Warthunder
# 15 - 24: Minecraft
# 24 - 33: Dirt Rally
scr_mean = [[], []]
tmp=[]
i=0
j=0
for x in cvx_eda_PC:
  j=0
  for y in x:
    tmp.append((np.mean(y[0]) - np.mean(cvx_eda_BS[i][j][0])) / np.std(cvx_eda_BS[i][j][0]))
    j+=1
  i+=1

print(tmp)
scr_mean[0] = tmp
tmp=[]
i=0
j=0
for x in cvx_eda_VR:
  j=0
  for y in x:
    tmp.append((np.mean(y[0]) - np.mean(cvx_eda_BS[i][j][0])) / np.std(cvx_eda_BS[i][j][0]))
    j+=1
  i+=1
scr_mean[1] = tmp

pc = scr_mean[0]
vr = scr_mean[1]
print("SCR")

print(f"FS: $\mu$: {np.mean(pc):.2f}, $\sigma$: {np.std(pc):.2f}")
print(f"VR: $\mu$: {np.mean(vr):.2f}, $\sigma$: {np.std(vr):.2f}")

normalancy_pc = stats.shapiro(pc)
normalancy_vr = stats.shapiro(vr)
print(f"FS: {normalancy_pc}")
print(f"VR: {normalancy_vr}")

result = stats.wilcoxon(pc, vr)
print(result)
print(f'Cohen-d: {cohen_d(pc, vr):.2f}')
print(f'Cliff\'s: {cliffs_delta(pc, vr):.2f}')

[8.908308374320173, 6.656392564204403, 0.012544785355730913, 3.145025204039363, 6.791807161656111, 0.0, -0.004079743122753311, 0.0, 0.047500887728772714, 1.6804046300336268, 0.011173551186930822, -0.5668127296638737, 0.0, -0.0045098615555116284, 0.008973075646150458, 0.06689496942505993, -0.3713069320939142, -0.4837596653797841, 0.1304648456860656, 0.22591009072554347, 27.547798161319644, 0.021993142204224093, 2.552390866050881, -0.5902047339634644, -0.046012846176576125, 0.717085592322033, -0.06989480107247704, -0.012900176511875678, 0.2623425717045436, 25.417615770392977, 0.11881972041940722, 31.42282144524508, -0.03727724183216073]
SCR
FS: $\mu$: 3.44, $\sigma$: 8.15
VR: $\mu$: 8.33, $\sigma$: 21.57
FS: ShapiroResult(statistic=0.5019033551216125, pvalue=1.8914303456796233e-09)
VR: ShapiroResult(statistic=0.43542730808258057, pvalue=3.778093393691506e-10)
WilcoxonResult(statistic=221.0, pvalue=0.29599490854889154)
Cohen-d: -0.30
Cliff's: 0.44


#SAM

In [ ]:
file = open('SAM.csv')

type(file)
csvreader = csv.reader(file)
sam = []
for row in csvreader:
  # print(row)
  sam.append(row)
Games=['LVLA', 'LVHA', 'HVHA', 'HVLA']

sam = pd.DataFrame(data=sam[1:],columns=sam[0]) 
# print(sam.dtypes)
sam["Arousal"] = sam["Arousal"].astype(str).astype(float)-1
sam["Valence"] = sam["Valence"].astype(str).astype(float)-1
sam["Dominance"] = 6-sam["Dominance"].astype(str).astype(float)
sam["CMA Quadrant"] = sam["Game"].astype(str)
print(sam)

   Participant ID  Game  Arousal  Valence  Dominance Type CMA Quadrant
0               1  HVHA      3.0      3.0        3.0   FS         HVHA
1               2  HVHA      2.0      4.0        1.0   VR         HVHA
2               3  HVHA      3.0      2.0        2.0   FS         HVHA
3               4  HVHA      4.0      4.0        3.0   VR         HVHA
4               5  HVHA      3.0      3.0        3.0   VR         HVHA
..            ...   ...      ...      ...        ...  ...          ...
61             62  LVLA      1.0      1.0        5.0   VR         LVLA
62             63  LVLA      2.0      3.0        3.0   FS         LVLA
63             64  LVLA      2.0      3.0        4.0   FS         LVLA
64             65  LVLA      1.0      1.0        4.0   VR         LVLA
65             66  LVLA      3.0      3.0        3.0   VR         LVLA

[66 rows x 7 columns]


In [ ]:
print(sam.query("Dominance>3 and Type=='VR'"))

   Participant ID  Game  Arousal  Valence  Dominance Type CMA Quadrant
12             13  HVHA      4.0      4.0        4.0   VR         HVHA
51             52  LVLA      0.0      1.0        4.0   VR         LVLA
53             54  LVLA      1.0      3.0        4.0   VR         LVLA
61             62  LVLA      1.0      1.0        5.0   VR         LVLA
64             65  LVLA      1.0      1.0        4.0   VR         LVLA


In [ ]:
print("D>=2")
print(sam.query("Dominance>=2").corr(method="spearman"))
print(stats.spearmanr(sam.query("Dominance>=2")["Arousal"], sam.query("Dominance>=2")["Dominance"]))
print(len(sam.query("Dominance>=2")["Arousal"]))
print("D<2")
print(sam.query("Dominance<=3").corr(method="spearman"))
print(stats.spearmanr(sam.query("Dominance<=3")["Arousal"], sam.query("Dominance<=3")["Dominance"]))
print("FS")
print(sam.query("Dominance>3 and Type == 'FS'").corr(method="spearman"))
print(len(sam.query("Dominance>3 and Type=='FS'")["Dominance"]))
print(stats.spearmanr(sam.query("Dominance>3 and Type=='FS'")["Arousal"], sam.query("Dominance>3  and Type=='FS'")["Dominance"]))
print("VR")
print(sam.query("Dominance>3 and Type == 'VR'").corr(method="spearman"))
print(len(sam.query("Dominance>3 and Type=='VR'")["Arousal"]))
print(stats.spearmanr(sam.query("Dominance>3 and Type=='VR'")["Arousal"], sam.query("Dominance>3  and Type=='VR'")["Dominance"]))


D>=2
            Arousal   Valence  Dominance
Arousal    1.000000  0.198148  -0.211898
Valence    0.198148  1.000000  -0.000447
Dominance -0.211898 -0.000447   1.000000
SignificanceResult(statistic=-0.21189798962036513, pvalue=0.12770583094784366)
53
D<2
            Arousal   Valence  Dominance
Arousal    1.000000  0.087332  -0.194777
Valence    0.087332  1.000000  -0.244189
Dominance -0.194777 -0.244189   1.000000
SignificanceResult(statistic=-0.1947769650320083, pvalue=0.16644828075745116)
FS
            Arousal   Valence  Dominance
Arousal    1.000000 -0.525485   0.582772
Valence   -0.525485  1.000000  -0.427121
Dominance  0.582772 -0.427121   1.000000
9
SignificanceResult(statistic=0.5827715174143585, pvalue=0.09959092002056928)
VR
           Arousal   Valence  Dominance
Arousal       1.00  0.750000   0.000000
Valence       0.75  1.000000  -0.395285
Dominance     0.00 -0.395285   1.000000
5
SignificanceResult(statistic=0.0, pvalue=1.0)


<ipython-input-85-4a499902adc6>:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(sam.query("Dominance>=2").corr(method="spearman"))
<ipython-input-85-4a499902adc6>:6: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(sam.query("Dominance<=3").corr(method="spearman"))
<ipython-input-85-4a499902adc6>:9: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(sam.query("Dominance>3 and Type == 'FS'").corr(method="spearman"))
<ipython-input-85-4a499902adc6>:13: FutureWarning: 

In [ ]:
for g in Games:
  x = sam.query(f'Game=="{g}"')
  # print(x)
  a=x["Arousal"].tolist()
  print(f"{g} Arousal Mdn: {np.median(a):.1f} ({np.percentile(a, 25):.1f} - {np.percentile(a, 75):.1f})")
  a=x["Valence"].tolist()
  print(f"{g} Valence Mdn: {np.median(a):.1f} ({np.percentile(a, 25):.1f} - {np.percentile(a, 75):.1f})")

LVLA Arousal Mdn: 1.0 (1.0 - 1.8)
LVLA Valence Mdn: 2.0 (1.2 - 3.0)
LVHA Arousal Mdn: 2.5 (2.0 - 4.0)
LVHA Valence Mdn: 2.0 (1.0 - 3.0)
HVHA Arousal Mdn: 3.0 (3.0 - 4.0)
HVHA Valence Mdn: 3.0 (2.0 - 3.8)
HVLA Arousal Mdn: 2.0 (1.0 - 2.8)
HVLA Valence Mdn: 3.0 (2.0 - 3.0)


In [ ]:
PC = sam.query(f'Type=="FS"')
VR = sam.query(f'Type=="VR"')

print("Arousal")
pc = PC['Arousal'].tolist()
vr = VR['Arousal'].tolist()
r = stats.wilcoxon(pc, vr)
print(f"FS: Mdn: {np.median(pc):.2f} ({np.percentile(pc, 25):.2f} - {np.percentile(pc, 75):.2f})")
print(f"FS: $\mu = $ {np.mean(pc):.1f} $\sigma = $ {np.std(pc):.1f}")
print(f"VR: Mdn: {np.median(vr):.2f} ({np.percentile(vr, 25):.2f} - {np.percentile(vr, 75):.2f})")
print(f"VR: $\mu = $ {np.mean(vr):.1f} $\sigma = $ {np.std(vr):.1f}")
print(f"W: {r[0]:.1f} p-value: {r[1]:.3f}")
print(f"d = {cohen_d(pc, vr)}")
print(f"cliff's = {cliffs_delta(pc, vr)}")

print("Valence")
pc = PC['Valence']
vr = VR['Valence']
r = stats.wilcoxon(pc, vr)
print(f"FS: Mdn: {np.median(pc):.2f} ({np.percentile(pc, 25):.2f} - {np.percentile(pc, 75):.2f})")
print(f"FS: $\mu = $ {np.mean(pc):.1f} $\sigma = $ {np.std(pc):.1f}")
print(f"VR: Mdn: {np.median(vr):.2f} ({np.percentile(vr, 25):.2f} - {np.percentile(vr, 75):.2f})")
print(f"VR: $\mu = $ {np.mean(vr):.1f} $\sigma = $ {np.std(vr):.1f}")
print(f"W: {r[0]:.1f} p-value: {r[1]:.3f}")
print(f"d = {cohen_d(pc, vr)}")
print(f"cliff's = {cliffs_delta(pc, vr)}")

print("Dominance")
pc = PC['Dominance']
vr = VR['Dominance']
r = stats.wilcoxon(pc, vr)
print(f"FS: Mdn: {np.median(pc):.2f} ({np.percentile(pc, 25):.2f} - {np.percentile(pc, 75):.2f})")
print(f"FS: $\mu = $ {np.mean(pc):.1f} $\sigma = $ {np.std(pc):.1f}")
print(f"VR: Mdn: {np.median(vr):.2f} ({np.percentile(vr, 25):.2f} - {np.percentile(vr, 75):.2f})")
print(f"VR: $\mu = $ {np.mean(vr):.1f} $\sigma = $ {np.std(vr):.1f}")
print(f"W: {r[0]:.1f} p-value: {r[1]:.3f}")
print(f"d = {cohen_d(pc, vr)}")
print(f"cliff's = {cliffs_delta(pc, vr)}")


Arousal
FS: Mdn: 2.00 (1.00 - 3.00)
FS: $\mu = $ 2.0 $\sigma = $ 1.0
VR: Mdn: 3.00 (2.00 - 4.00)
VR: $\mu = $ 2.6 $\sigma = $ 1.3
W: 66.5 p-value: 0.007
d = -0.5261057786980897
cliff's = 0.2516069788797062
Valence
FS: Mdn: 2.00 (2.00 - 3.00)
FS: $\mu = $ 2.3 $\sigma = $ 0.9
VR: Mdn: 3.00 (2.00 - 4.00)
VR: $\mu = $ 2.6 $\sigma = $ 1.2
W: 75.5 p-value: 0.262
d = -0.308732022447085
cliff's = 0.2773186409550046
Dominance
FS: Mdn: 2.00 (1.00 - 3.00)
FS: $\mu = $ 1.8 $\sigma = $ 1.0
VR: Mdn: 1.00 (0.00 - 2.00)
VR: $\mu = $ 1.3 $\sigma = $ 1.1
W: 88.0 p-value: 0.020
d = 0.5398835767892611
cliff's = 0.5381083562901745


/usr/local/lib/python3.10/dist-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


In [ ]:
x = sam.query(f'Type=="VR"')
y = sam.query(f'Type=="FS"')
rx=np.sqrt(np.square(x['Arousal'].tolist()) + np.square(x['Valence'].tolist()))
ry=np.sqrt(np.square(y['Arousal'].tolist()) + np.square(y['Valence'].tolist()))
print(f'VR: $\mu$ {np.mean(rx):.2f}, $\sigma$ {np.std(rx):.2f}')
print(f'FS: $\mu$ {np.mean(ry):.2f}, $\sigma$ {np.std(ry):.2f}')
print(stats.shapiro(rx))
print(stats.shapiro(ry))
r = stats.wilcoxon(rx, ry)
print(r)
print(f"d = {cohen_d(rx, ry): .2f}")
print(f"cliff's = {cliffs_delta(rx, ry):.2f}")

VR: $\mu$ 3.92, $\sigma$ 1.32
FS: $\mu$ 3.22, $\sigma$ 0.91
ShapiroResult(statistic=0.9017431735992432, pvalue=0.005913454573601484)
ShapiroResult(statistic=0.9512921571731567, pvalue=0.14509834349155426)
WilcoxonResult(statistic=89.5, pvalue=0.003263746056570931)
d =  0.61
cliff's = 0.54


/usr/local/lib/python3.10/dist-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


In [ ]:
print("Arousal")
pc = PC['Arousal'].tolist()
vr = VR['Arousal'].tolist()
r = stats.wilcoxon(pc, vr)
print(f"FS: Mdn: {np.median(pc):.2f} ({np.percentile(pc, 25):.2f} - {np.percentile(pc, 75):.2f})")
print(f"FS: $\mu = $ {np.mean(pc):.1f} $\sigma = $ {np.std(pc):.1f}")
print(f"VR: Mdn: {np.median(vr):.2f} ({np.percentile(vr, 25):.2f} - {np.percentile(vr, 75):.2f})")
print(f"VR: $\mu = $ {np.mean(vr):.1f} $\sigma = $ {np.std(vr):.1f}")
print(f"W: {r[0]:.1f} p-value: {r[1]:.3f}")
print(f"d = {cohen_d(pc, vr)}")
print(f"cliff's = {cliffs_delta(pc, vr)}\n")

print("Valence")
pc = PC['Valence']
vr = VR['Valence']
r = stats.wilcoxon(pc, vr)
print(f"FS: Mdn: {np.median(pc):.2f} ({np.percentile(pc, 25):.2f} - {np.percentile(pc, 75):.2f})")
print(f"FS: $\mu = $ {np.mean(pc):.1f} $\sigma = $ {np.std(pc):.1f}")
print(f"VR: Mdn: {np.median(vr):.2f} ({np.percentile(vr, 25):.2f} - {np.percentile(vr, 75):.2f})")
print(f"VR: $\mu = $ {np.mean(vr):.1f} $\sigma = $ {np.std(vr):.1f}")
print(f"W: {r[0]:.1f} p-value: {r[1]:.3f}")
print(f"d = {cohen_d(pc, vr)}")
print(f"cliff's = {cliffs_delta(pc, vr)}\n")

print("Dominance")
pc = PC['Dominance']
vr = VR['Dominance']
r = stats.wilcoxon(pc, vr)
print(f"FS: Mdn: {np.median(pc):.2f} ({np.percentile(pc, 25):.2f} - {np.percentile(pc, 75):.2f})")
print(f"FS: $\mu = $ {np.mean(pc):.1f} $\sigma = $ {np.std(pc):.1f}")
print(f"VR: Mdn: {np.median(vr):.2f} ({np.percentile(vr, 25):.2f} - {np.percentile(vr, 75):.2f})")
print(f"VR: $\mu = $ {np.mean(vr):.1f} $\sigma = $ {np.std(vr):.1f}")
print(f"W: {r[0]:.1f} p-value: {r[1]:.3f}")
print(f"d = {cohen_d(pc, vr)}")
print(f"cliff's = {cliffs_delta(pc, vr)}")

Arousal
FS: Mdn: 2.00 (1.00 - 3.00)
FS: $\mu = $ 2.0 $\sigma = $ 1.0
VR: Mdn: 3.00 (2.00 - 4.00)
VR: $\mu = $ 2.6 $\sigma = $ 1.3
W: 66.5 p-value: 0.007
d = -0.5261057786980897
cliff's = 0.2516069788797062

Valence
FS: Mdn: 2.00 (2.00 - 3.00)
FS: $\mu = $ 2.3 $\sigma = $ 0.9
VR: Mdn: 3.00 (2.00 - 4.00)
VR: $\mu = $ 2.6 $\sigma = $ 1.2
W: 75.5 p-value: 0.262
d = -0.308732022447085
cliff's = 0.2773186409550046

Dominance
FS: Mdn: 2.00 (1.00 - 3.00)
FS: $\mu = $ 1.8 $\sigma = $ 1.0
VR: Mdn: 1.00 (0.00 - 2.00)
VR: $\mu = $ 1.3 $\sigma = $ 1.1
W: 88.0 p-value: 0.020
d = 0.5398835767892611
cliff's = 0.5381083562901745


/usr/local/lib/python3.10/dist-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


In [ ]:
import scikit_posthocs as sp
# PC-VR-Mix
Games=['LVLA', 'LVHA', 'HVHA', 'HVLA']
arousal=[[], [], []]
valence=[[], [], []]

bygame=[[], [], [], []]
bygame[0] = sam[sam['Game'] == 'LVLA']
bygame[1] = sam[sam['Game'] == 'LVHA']
bygame[2] = sam[sam['Game'] == 'HVHA']
bygame[3] = sam[sam['Game'] == 'HVLA']

for (i, df) in enumerate(bygame):
  # print(Games[i])

  # Mix
  # print("")
  ar = df['Arousal'].tolist()
  vl = df['Valence'].tolist()
  # print(f"{np.mean(ar)} {np.std(ar)}")
  # print(f"{np.mean(vl)} {np.std(vl)}")
  arousal[2].append(ar)
  valence[2].append(vl)

  ar = df.query('Type=="FS"')['Arousal'].tolist()
  vl = df.query('Type=="FS"')['Valence'].tolist()
  arousal[0].append(ar)
  valence[0].append(vl)

  ar = df.query('Type=="VR"')['Arousal'].tolist()
  vl = df.query('Type=="VR"')['Valence'].tolist()
  arousal[1].append(ar)
  valence[1].append(vl)

for i in range(2,3):
  print(["PC", "VR", "All"][i])
  print("Arousal")
  result = stats.kruskal(arousal[i][0], arousal[i][1], arousal[i][2], arousal[i][3])
  print(result)
  

  print("Valence")
  result = stats.kruskal(valence[i][0], valence[i][1], valence[i][2], valence[i][3])
  print(result)
  print()
  # for j in range(4):
  #   print(Games[j])
  #   print(f"Arousal: M {np.mean(arousal[i][j]):0.2f}, SD {np.std(arousal[i][j]):0.2f}")
  #   print(f"Valence: M {np.mean(valence[i][j]):0.2f}, SD {np.std(valence[i][j]):0.2f}")
  
  print("Post hoc: ")
  adf = sam[['Arousal', 'CMA Quadrant']].copy()
  print("\nArousal")
  ph = sp.posthoc_dunn(sam, group_col='CMA Quadrant', val_col='Arousal')
  print(ph)
  print("\nValence")
  ph = sp.posthoc_dunn(sam, group_col='CMA Quadrant', val_col='Arousal')
  print(ph)

All
Arousal
KruskalResult(statistic=30.671288942014424, pvalue=9.96883973452592e-07)
Valence
KruskalResult(statistic=4.75557827232003, pvalue=0.19059511595420348)

Post hoc: 

Arousal
              HVHA      HVLA      LVHA          LVLA
HVHA  1.000000e+00  0.002271  0.156609  1.364451e-07
HVLA  2.271275e-03  1.000000  0.100571  6.047019e-02
LVHA  1.566091e-01  0.100571  1.000000  2.189760e-04
LVLA  1.364451e-07  0.060470  0.000219  1.000000e+00

Valence
              HVHA      HVLA      LVHA          LVLA
HVHA  1.000000e+00  0.002271  0.156609  1.364451e-07
HVLA  2.271275e-03  1.000000  0.100571  6.047019e-02
LVHA  1.566091e-01  0.100571  1.000000  2.189760e-04
LVLA  1.364451e-07  0.060470  0.000219  1.000000e+00


#VAS

In [ ]:
file = open('VAS.csv')

type(file)
csvreader = csv.reader(file)
vas = []
for row in csvreader:
  # print(row)
  vas.append(row)

vas = pd.DataFrame(data=vas[1:],columns=vas[0]) 
vas["CMA Quadrant"] = vas["Game"].astype(str)

In [ ]:
columns = ['Joy', 'Anger', 'Calmness', 'Sadness', 'Disgust', 'Relaxation', 'Happiness', 'Fear', 'Anxiousness', 'Dizziness']

for c in columns:
  vas[c] = vas[c].astype(str).astype(float)
print(vas.dtypes)
print(vas.head())

Id               object
Game             object
Dropdown         object
Joy             float64
Anger           float64
Calmness        float64
Sadness         float64
Disgust         float64
Relaxation      float64
Happiness       float64
Fear            float64
Anxiousness     float64
Dizziness       float64
CMA Quadrant     object
dtype: object
  Id  Game Dropdown   Joy  Anger  Calmness  Sadness  Disgust  Relaxation  \
0  1  HVHA       VR  81.0   14.0      35.0     16.0     59.0        53.0   
1  2  HVHA       FS  75.0    0.0      54.0     39.0     33.0        52.0   
2  3  HVHA       FS  69.0    0.0      28.0      0.0      0.0         7.0   
3  4  HVHA       VR  71.0    0.0       4.0      0.0     15.0         0.0   
4  5  HVHA       FS  28.0    0.0      54.0      0.0     46.0        32.0   

   Happiness  Fear  Anxiousness  Dizziness CMA Quadrant  
0       85.0  16.0         16.0        0.0         HVHA  
1       86.0   6.0         16.0        0.0         HVHA  
2       52.0   0.0 

In [ ]:
arr=[]
columns = ['Joy', 'Anger', 'Calmness', 'Sadness', 'Disgust', 'Relaxation', 'Happiness', 'Fear', 'Anxiousness', 'Dizziness']
games = ["HVHA", "HVLA", "LVHA", "LVLA"]
PC = vas.query('Dropdown=="FS"')
VR = vas.query('Dropdown=="VR"')

for g in games:
  # print(g)
  pcg = PC[PC['Game'] == g]
  vrg = VR[VR['Game'] == g]
  for c in columns:
    # print(c)
    df = pd.DataFrame(pcg[c].tolist(), columns = ["Val"])
    df["Game"] = g
    df["Emotion"] = c
    df["Type"] = "FS"
    arr.append(df)
    df = pd.DataFrame(vrg[c].tolist(), columns = ["Val"])
    df["Game"] = g
    df["Emotion"] = c
    df["Type"] = "VR"
    arr.append(df)
    # print(c)

df =  pd.concat(arr, ignore_index=True)
print(df)

      Val  Game    Emotion Type
0    75.0  HVHA        Joy   FS
1    69.0  HVHA        Joy   FS
2    28.0  HVHA        Joy   FS
3    47.0  HVHA        Joy   FS
4    50.0  HVHA        Joy   FS
..    ...   ...        ...  ...
655  38.0  LVLA  Dizziness   VR
656  49.0  LVLA  Dizziness   VR
657  65.0  LVLA  Dizziness   VR
658  17.0  LVLA  Dizziness   VR
659  30.0  LVLA  Dizziness   VR

[660 rows x 4 columns]


In [ ]:
for e in columns:
  # print(e)
  d = df.query(f'Emotion=="{e}"')
  x = d.query('Type=="VR"')["Val"].tolist()
  y = d.query('Type=="FS"')["Val"].tolist()
  result=stats.wilcoxon(x, y)
  print(f"{e}: W: {result[0]:.2f}, p-value: {result[1]:.4f}")
  print(f"d = {cohen_d(x, y): .2f}")
  print(f"Cliffs = {cliffs_delta(x, y): .2f}\n")

Joy: W: 101.50, p-value: 0.0041
d =  0.56
Cliffs =  0.51
Anger: W: 171.50, p-value: 0.3198
d =  0.20
Cliffs =  0.46
Calmness: W: 107.50, p-value: 0.1389
d = -0.19
Cliffs =  0.43
Sadness: W: 48.00, p-value: 0.3007
d = -0.12
Cliffs =  0.26
Disgust: W: 50.50, p-value: 0.2176
d =  0.11
Cliffs =  0.37
Relaxation: W: 93.00, p-value: 0.1708
d = -0.18
Cliffs =  0.35
Happiness: W: 174.00, p-value: 0.1469
d =  0.21
Cliffs =  0.43
Fear: W: 184.00, p-value: 0.2096
d =  0.27
Cliffs =  0.47
Anxiousness: W: 260.50, p-value: 0.9478
d =  0.08
Cliffs =  0.49
Dizziness: W: 8.50, p-value: 0.0000
d =  1.25
Cliffs =  0.53
